In [1]:
!pip install transformers datasets scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
%cd /content/drive/MyDrive/ANLP_indiv_project

/content/drive/MyDrive/ANLP_indiv_project


In [4]:
!pwd

/content/drive/MyDrive/ANLP_indiv_project


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import torch

# Load the model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Function to preprocess the dataset
def preprocess(data):
    return tokenizer(data['text'], padding='max_length', truncation=True)

# Function to compute the F1 score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    return {"f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Load datasets (example)
dataset_files = [
    "datasets/processed_DAVIDSON.csv",
    "datasets/processed_DYNABENCH.csv",
    "datasets/processed_MHS.csv",
    "datasets/processed_MLMA.csv"
]

all_f1_scores = []

for file in dataset_files:
    # Load and preprocess the dataset
    dataset = pd.read_csv(file)
    dataset = Dataset.from_pandas(dataset)
    dataset = dataset.map(preprocess, batched=True)
    dataset = dataset.train_test_split(test_size=0.2)

    # 5-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []

    # Re-load model for each dataset
    model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

    for fold, (train_index, val_index) in enumerate(kf.split(dataset['train'])):
        train_dataset = dataset['train'].select(train_index)
        val_dataset = dataset['train'].select(val_index)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        # Train the model
        trainer.train()

        # Evaluate the model
        eval_result = trainer.evaluate()
        f1_scores.append(eval_result['eval_f1'])

    mean_f1 = sum(f1_scores) / len(f1_scores)
    all_f1_scores.append(mean_f1)
    print(f"Mean F1 Score for {file}: {mean_f1}")

    # Save the model after all folds
    model_save_path = f'./models/{os.path.basename(file).split(".")[0]}_final'
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

overall_mean_f1 = sum(all_f1_scores) / len(all_f1_scores)
print(f"Overall Mean F1 Score: {overall_mean_f1}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/24783 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.756700
20,0.727900
30,0.711200
40,0.654500
50,0.573500
60,0.470200
70,0.425000
80,0.529700
90,0.350600
100,0.443600


Step,Training Loss
10,0.060900
20,0.083400
30,0.045500
40,0.084500
50,0.108800
60,0.208500
70,0.070300
80,0.036200
90,0.043900
100,0.114000


Step,Training Loss
10,0.201000
20,0.119800
30,0.017200
40,0.087400
50,0.133100
60,0.084300
70,0.105100
80,0.082400
90,0.007200
100,0.061400
